<a href="https://colab.research.google.com/github/Sathvika2023/MIniproject/blob/main/01_Eshara_datapreopenpose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install required Python packages
!pip install -q numpy==1.23.5 opencv-python tqdm mediapipe tensorflow==2.12.0 moviepy gdown

# Optional: install unzip utility (only needed if you're downloading zip files)
!apt-get -qq install -y unzip
# Install the correct version of NumPy (if it’s not already installed)
!pip install numpy==1.23.5

# Uninstall the current TensorFlow installation
!pip uninstall tensorflow

# Reinstall TensorFlow with the correct NumPy version linked
!pip install tensorflow==2.12.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 129.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 127.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires proto

In [ ]:

# Cell 3: Imports
import os
import cv2
import numpy as np
from tqdm import tqdm
from moviepy.editor import VideoFileClip
import mediapipe as mp
from sklearn.model_selection import train_test_split


  if event.key is 'enter':



In [ ]:

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=2,
    enable_segmentation=False,
    min_detection_confidence=0.5)

In [ ]:

# Configuration
DATASET_PATH = "/content/drive/MyDrive/datauk.zip"  # Update this
OUTPUT_PATH = "/content/drive/MyDrive/processed_openpose_data"
IMG_SIZE = (368, 368)  # OpenPose standard input size
HEATMAP_SIZE = (46, 46)  # 1/8 of input size
SEQ_LENGTH = 16  # For temporal processing
os.makedirs(OUTPUT_PATH, exist_ok=True)

In [ ]:

import os
import cv2
import zipfile
from moviepy.editor import VideoFileClip
from tqdm import tqdm
import mediapipe as mp
import sys
from pathlib import Path

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False,
                   min_detection_confidence=0.5,
                   min_tracking_confidence=0.5)

def process_videos_to_openpose(video_dir, output_dir, frames_per_video=50):
    """Extracts frames and OpenPose keypoints from videos, handling nested zip structures"""
    os.makedirs(output_dir, exist_ok=True)
    all_frames = []
    all_keypoints = []
    processed_words = set()

    print(f"📂 Scanning: {video_dir}")
    sys.stdout.flush()

    def process_video_file(video_path, word_category=""):
        """Helper function to process individual video files"""
        try:
            clip = VideoFileClip(video_path)
            video_name = Path(video_path).stem

            # Create output subdirectory for word category
            word_output_dir = os.path.join(output_dir, word_category)
            os.makedirs(word_output_dir, exist_ok=True)

            frames = []
            keypoints = []

            for frame_idx, frame in enumerate(clip.iter_frames()):
                if frame_idx % max(1, int(clip.fps / frames_per_video)) == 0:
                    # Save frame
                    frame_path = os.path.join(word_output_dir,
                                            f"{word_category}_{video_name}_{frame_idx}.jpg")
                    success = cv2.imwrite(frame_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

                    if not success:
                        print(f"❌ Failed to save: {frame_path}")
                        continue

                    frames.append(frame_path)

                    # Get keypoints
                    results = pose.process(frame)
                    if results.pose_landmarks:
                        kps = []
                        for landmark in results.pose_landmarks.landmark:
                            kps.extend([landmark.x, landmark.y, landmark.visibility])
                        keypoints.append(kps)
                    else:
                        keypoints.append([])

            clip.close()
            return frames, keypoints

        except Exception as e:
            print(f"⚠️ Error processing {video_path}: {str(e)}")
            return [], []

    if video_dir.endswith(".zip"):
        with zipfile.ZipFile(video_dir, 'r') as zip_ref:
            # Find all video files, preserving directory structure
            video_files = [f for f in zip_ref.namelist()
                         if f.lower().endswith(('.mp4', '.avi', '.mov'))]

            print(f"🎞 Found {len(video_files)} videos in zip")

            for video_file in tqdm(video_files, desc="Processing videos"):
                # Extract word category from path (assuming structure: words/word1/videos.mp4)
                path_parts = Path(video_file).parts
                word_category = path_parts[-2] if len(path_parts) > 1 else "unknown"

                if word_category not in processed_words:
                    print(f"\nProcessing word category: {word_category}")
                    processed_words.add(word_category)

                # Extract and process video
                with zip_ref.open(video_file) as video_data:
                    temp_path = f"temp_{Path(video_file).name}"
                    with open(temp_path, "wb") as temp_file:
                        temp_file.write(video_data.read())

                    frames, kps = process_video_file(temp_path, word_category)
                    all_frames.extend(frames)
                    all_keypoints.extend(kps)

                    # Clean up
                    if os.path.exists(temp_path):
                        os.remove(temp_path)
    else:
        # Process directory with word subfolders
        for root, dirs, files in os.walk(video_dir):
            video_files = [f for f in files if f.lower().endswith(('.mp4', '.avi', '.mov'))]

            if video_files:
                word_category = Path(root).name
                print(f"\nProcessing word category: {word_category}")

                for video_file in tqdm(video_files, desc=f"Processing {word_category}"):
                    video_path = os.path.join(root, video_file)
                    frames, kps = process_video_file(video_path, word_category)
                    all_frames.extend(frames)
                    all_keypoints.extend(kps)

    print(f"\n✅ Done. Extracted {len(all_frames)} frames and {len(all_keypoints)} keypoints sets.")
    print(f"Processed {len(processed_words)} word categories: {sorted(processed_words)}")

    return all_frames, all_keypoints

# Example usage:
# frames, keypoints = process_videos_to_openpose("dataset.zip", "output_frames")
# OR
# frames, keypoints = process_videos_to_openpose("dataset_folder", "output_frames")

In [ ]:

# Cell 5: Heatmap Generator
def generate_heatmaps(keypoints_list):
    """Converts keypoints to Gaussian heatmaps"""
    heatmaps = []
    for kps in keypoints_list:
        heatmap = np.zeros((*HEATMAP_SIZE, len(kps)//3), dtype=np.float32)

        for i in range(0, len(kps), 3):
            x, y, v = kps[i], kps[i+1], kps[i+2]
            if v > 0.1:  # Only visible keypoints
                x_hm = int(x * HEATMAP_SIZE[1])
                y_hm = int(y * HEATMAP_SIZE[0])

                # Create 2D Gaussian
                xx, yy = np.meshgrid(np.arange(HEATMAP_SIZE[1]), np.arange(HEATMAP_SIZE[0]))
                heatmap[..., i//3] = np.exp(-((xx - x_hm)**2 + (yy - y_hm)**2) / (2 * 2.5**2))

        heatmaps.append(heatmap)
    return heatmaps

In [ ]:


# Cell 6: Data Augmentation
def augment_data(image, heatmap):
    """Applies random augmentations"""
    # Random flip
    if np.random.rand() > 0.5:
        image = cv2.flip(image, 1)
        heatmap = np.flip(heatmap, axis=1)

    # Random brightness
    alpha = np.random.uniform(0.8, 1.2)
    image = cv2.convertScaleAbs(image, alpha=alpha)

    return image, heatmap


In [ ]:
# Cell 7: Sequence Generator for CNN+RNN
import tensorflow as tf # Import tensorflow before using it
SEQ_LENGTH = 16
class OpenPoseSequenceGenerator(tf.keras.utils.Sequence):
    def __init__(self, frame_paths, keypoints, batch_size=8, seq_length=SEQ_LENGTH, augment=True):
        self.frame_paths = frame_paths
        self.keypoints = keypoints
        self.batch_size = batch_size
        self.seq_length = seq_length
        self.augment = augment
        self.indices = np.arange(len(frame_paths) - seq_length + 1)

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, idx):
        batch_indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]

        batch_images = []
        batch_heatmaps = []

        for i in batch_indices:
            sequence_images = []
            sequence_heatmaps = []

            for j in range(i, i+self.seq_length):
                # Load frame
                img = cv2.imread(self.frame_paths[j])
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, IMG_SIZE).astype(np.float32)/255.0

                # Generate heatmap
                heatmap = np.zeros((*HEATMAP_SIZE, len(self.keypoints[j])//3), dtype=np.float32)
                for k in range(0, len(self.keypoints[j]), 3):
                    x, y, v = self.keypoints[j][k], self.keypoints[j][k+1], self.keypoints[j][k+2]
                    if v > 0.1:
                        x_hm = int(x * HEATMAP_SIZE[1])
                        y_hm = int(y * HEATMAP_SIZE[0])
                        heatmap[y_hm-1:y_hm+2, x_hm-1:x_hm+2, k//3] = 1.0

                # Augment if training
                if self.augment:
                    img, heatmap = augment_data(img, heatmap)

                sequence_images.append(img)
                sequence_heatmaps.append(heatmap)

            batch_images.append(np.stack(sequence_images))
            batch_heatmaps.append(np.stack(sequence_heatmaps))

        return np.array(batch_images), np.array(batch_heatmaps)

    def on_epoch_end(self):
        np.random.shuffle(self.indices)

In [ ]:
import zipfile
# Cell 8: Main Processing Pipeline
def main():
    # 1. Process videos to frames + keypoints
    print("Processing videos to OpenPose data...")
    frame_paths, keypoints = process_videos_to_openpose(
        DATASET_PATH,
        os.path.join(OUTPUT_PATH, "frames")
    )

    # Check if frame_paths is empty
    if not frame_paths:
        print("Error: No frames extracted. Check your dataset path and video format.")
        return  # Exit the function if no frames are found

    # 2. Split data (video-wise)
    video_names = list(set([os.path.basename(os.path.dirname(f)) for f in frame_paths]))
    train_vids, test_vids = train_test_split(video_names, test_size=0.2, random_state=42)

    train_data = [(f, k) for f, k in zip(frame_paths, keypoints)
                 if os.path.basename(os.path.dirname(f)) in train_vids]
    test_data = [(f, k) for f, k in zip(frame_paths, keypoints)
                if os.path.basename(os.path.dirname(f)) in test_vids]

    # 3. Create generators
    train_gen = OpenPoseSequenceGenerator(
        [d[0] for d in train_data],
        [d[1] for d in train_data],
        augment=True
    )
    test_gen = OpenPoseSequenceGenerator(
        [d[0] for d in test_data],
        [d[1] for d in test_data],
        augment=False
    )

    # 4. Verify output
    sample_images, sample_heatmaps = train_gen[0]
    print("\nSample batch shapes:")
    print(f"Images: {sample_images.shape}")  # (batch, seq_len, 368, 368, 3)
    print(f"Heatmaps: {sample_heatmaps.shape}")  # (batch, seq_len, 46, 46, 25)

    # 5. Save metadata
    np.savez(os.path.join(OUTPUT_PATH, "dataset_meta.npz"),
             train_frames=[d[0] for d in train_data],
             test_frames=[d[0] for d in test_data],
             input_shape=sample_images.shape[1:],
             output_shape=sample_heatmaps.shape[1:])

    print(f"\nProcessing complete! Data saved to {OUTPUT_PATH}")

if __name__ == "__main__":
    main()

Processing videos to OpenPose data...
📂 Scanning: /content/drive/MyDrive/datauk.zip
🎞 Found 730 videos in zip


Processing videos:   0%|          | 0/730 [00:00<?, ?it/s]


Processing word category: 1. loud


  warnings.warn("Warning: in file %s, "%(self.filename)+

Processing videos:   2%|▏         | 15/730 [07:14<4:48:07, 24.18s/it]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file temp_MVI_9448.MOV, 6220800 bytes wanted but 0 bytes read,at frame 56/57, at time 2.24/2.24 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

Processing videos:   2%|▏         | 18/730 [08:26<4:44:54, 24.01s/it]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file temp_MVI_9534.MOV, 6220800 bytes wanted but 0 bytes read,at frame 56/57, at time 2.24/2.24 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

Processing videos:   3%|▎         | 21/730 [09:52<5:18:20, 26.94s/it]


Processing word category: 2. quiet


...


Processing word category: 98. sick


Processing videos:  95%|█████████▌| 697/730 [6:08:36<15:06, 27.47s/it]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file temp_MVI_9280.MOV, 6220800 bytes wanted but 0 bytes read,at frame 56/57, at time 2.24/2.24 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

Processing videos:  97%|█████████▋| 705/730 [6:12:13<11:25, 27.40s/it]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file temp_MVI_9444.MOV, 6220800 bytes wanted but 0 bytes read,at frame 56/57, at time 2.24/2.24 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

Processing videos:  97%|█████████▋| 709/730 [6:14:20<10:52, 31.05s/it]


Processing word category: 99. healthy


Processing videos: 100%|██████████| 730/730 [6:24:54<00:00, 31.64s/it]



✅ Done. Extracted 45295 frames and 45295 keypoints sets.
Processed 41 word categories: ['1. loud', '2. quiet', '3. happy', '36. light', '39. famous', '4. sad', '40. I', '41. you', '42. he', '43. she', '44. it', '45. we', '46. you (plural)', '47. they', '48. Hello', '49. How are you', '5. Beautiful', '50. Alright', '51. Good Morning', '52. Good afternoon', '53. Good evening', '54. Good night', '55. Thank you', '56. Pleased', '6. Ugly', '7. Deaf', '78. long', '79. short', '8. Blind', '80. tall', '81. wide', '87. hot', '88. cold', '89. warm', '90. cool', '91. new', '97. dry', '98. sick', '99. healthy', 'Extra', 'extra']

Sample batch shapes:
Images: (8, 16, 368, 368, 3)
Heatmaps: (8, 16, 46, 46, 33)

Processing complete! Data saved to /content/drive/MyDrive/processed_openpose_data


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path

In [ ]:
# Configuration
PROCESSED_PATH = Path("/content/drive/MyDrive/processed_openpose_data")
OUTPUT_ROOT = Path("/content/drive/MyDrive/openpose_train_test_data")
TEST_SIZE = 0.2
SEED = 42

# Augmentation configuration
TRAIN_AUGMENTOR = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.9, 1.1],
    fill_mode='constant'
)

In [ ]:
import os
import shutil
from pathlib import Path
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor

# ========== CONFIG ==========
PROCESSED_PATH = Path("/content/drive/MyDrive/processed_openpose_data")  # Use local path, not Google Drive
OUTPUT_ROOT = Path("/content/drive/MyDrive/openpose_train_test_data")
TEST_SIZE = 0.2
SEED = 42

# Dummy augmentation (replace with your real augmenter)
class DummyAugmentor:
    def random_transform(self, img):
        return cv2.flip(img, 1)  # Simple horizontal flip

TRAIN_AUGMENTOR = DummyAugmentor()

# ========== MAIN ==========
def process_and_split():
    all_samples = []
    for root, _, files in os.walk(PROCESSED_PATH):
        if not files:
            continue
        category = Path(root).name
        for file in files:
            if file.endswith(('.jpg', '.png')):
                all_samples.append((Path(root) / file, category))

    print(f"Found {len(all_samples)} images in {PROCESSED_PATH}")

    file_paths = [str(p[0]) for p in all_samples]
    categories = [p[1] for p in all_samples]

    train_files, test_files = train_test_split(
        file_paths,
        test_size=TEST_SIZE,
        random_state=SEED,
        stratify=categories
    )

    print(f"Train: {len(train_files)} | Test: {len(test_files)}")

    train_dir = OUTPUT_ROOT / "train"
    test_dir = OUTPUT_ROOT / "test"
    train_dir.mkdir(parents=True, exist_ok=True)
    test_dir.mkdir(parents=True, exist_ok=True)

    def process_one(src_path, target_dir, augment):
        src = Path(src_path)
        category = src.parent.name
        dest_dir = target_dir / category
        dest_dir.mkdir(exist_ok=True)

        # Copy original image
        dest_file = dest_dir / src.name
        if not dest_file.exists():
            shutil.copy2(src, dest_file)

        # Augment
        if augment:
            img = cv2.imread(str(src))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            for i in range(1):  # 1 augmented image per original
                aug_img = TRAIN_AUGMENTOR.random_transform(img)
                aug_file = dest_dir / f"aug_{i}_{src.name}"
                cv2.imwrite(str(aug_file), cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR))

    def process_split_parallel(file_list, target_dir, augment=False):
        with ThreadPoolExecutor(max_workers=8) as executor:
            list(tqdm(executor.map(lambda f: process_one(f, target_dir, augment), file_list), total=len(file_list)))

    # Process both splits
    process_split_parallel(train_files, train_dir, augment=True)
    process_split_parallel(test_files, test_dir, augment=False)

    print("✅ Processing complete.")
    print(f"Train files: {sum(len(f) for _, _, f in os.walk(train_dir))}")
    print(f"Test files: {sum(len(f) for _, _, f in os.walk(test_dir))}")

# Run the processing
if __name__ == "__main__":
    process_and_split()


Found 45295 images in /content/drive/MyDrive/processed_openpose_data
Train: 36236 | Test: 9059


100%|██████████| 9059/9059 [08:32<00:00, 17.68it/s]


✅ Processing complete.
Train files: 103952
Test files: 9059


In [ ]:
import numpy as np

# Load the .npz file
data = np.load('/content/drive/MyDrive/processed_openpose_data/dataset_meta.npz', allow_pickle=True)

# List all keys in the file
print("Keys in the .npz file:", data.files)

# Example: Inspect the content of each key
for key in data.files:
    print(f"\nKey: {key}")
    print(data[key])


Keys in the .npz file: ['train_frames', 'test_frames', 'input_shape', 'output_shape']

Key: train_frames
['/content/drive/MyDrive/processed_openpose_data/frames/1. loud/1. loud_temp_MVI_5177_0.jpg'
 '/content/drive/MyDrive/processed_openpose_data/frames/1. loud/1. loud_temp_MVI_5177_1.jpg'
 '/content/drive/MyDrive/processed_openpose_data/frames/1. loud/1. loud_temp_MVI_5177_2.jpg'
 ...
 '/content/drive/MyDrive/processed_openpose_data/frames/99. healthy/99. healthy_temp_MVI_9532_59.jpg'
 '/content/drive/MyDrive/processed_openpose_data/frames/99. healthy/99. healthy_temp_MVI_9532_60.jpg'
 '/content/drive/MyDrive/processed_openpose_data/frames/99. healthy/99. healthy_temp_MVI_9532_61.jpg']

Key: test_frames
['/content/drive/MyDrive/processed_openpose_data/frames/48. Hello/48. Hello_temp_MVI_0029_0.jpg'
 '/content/drive/MyDrive/processed_openpose_data/frames/48. Hello/48. Hello_temp_MVI_0029_1.jpg'
 '/content/drive/MyDrive/processed_openpose_data/frames/48. Hello/48. Hello_temp_MVI_0029_2.

In [ ]:
import numpy as np

# Path where keypoints are saved
keypoint_file = "/content/drive/MyDrive/processed_openpose_data/dataset_meta.npz"

# Load the .npz file
data = np.load(keypoint_file, allow_pickle=True)

# List all keys in the file
print("Keys in the .npz file:", data.files)

# Example: Inspect the content of each key
for key in data.files:
    print(f"\nKey: {key}")
    print(data[key])

Keys in the .npz file: ['train_frames', 'test_frames', 'input_shape', 'output_shape']

Key: train_frames
['/content/drive/MyDrive/processed_openpose_data/frames/1. loud/1. loud_temp_MVI_5177_0.jpg'
 '/content/drive/MyDrive/processed_openpose_data/frames/1. loud/1. loud_temp_MVI_5177_1.jpg'
 '/content/drive/MyDrive/processed_openpose_data/frames/1. loud/1. loud_temp_MVI_5177_2.jpg'
 ...
 '/content/drive/MyDrive/processed_openpose_data/frames/99. healthy/99. healthy_temp_MVI_9532_59.jpg'
 '/content/drive/MyDrive/processed_openpose_data/frames/99. healthy/99. healthy_temp_MVI_9532_60.jpg'
 '/content/drive/MyDrive/processed_openpose_data/frames/99. healthy/99. healthy_temp_MVI_9532_61.jpg']

Key: test_frames
['/content/drive/MyDrive/processed_openpose_data/frames/48. Hello/48. Hello_temp_MVI_0029_0.jpg'
 '/content/drive/MyDrive/processed_openpose_data/frames/48. Hello/48. Hello_temp_MVI_0029_1.jpg'
 '/content/drive/MyDrive/processed_openpose_data/frames/48. Hello/48. Hello_temp_MVI_0029_2.

In [ ]:
for key in data.files:
    print("Key found:", key)


Key found: train_frames
Key found: test_frames
Key found: input_shape
Key found: output_shape


In [ ]:
print("Train shape:", data['train_frames'].shape)
print("Test shape:", data['test_frames'].shape)


Train shape: (35819,)
Test shape: (9476,)


In [ ]:
print("Type of train_frames:", type(data['train_frames']))
print("First item:", data['train_frames'][0])


Type of train_frames: <class 'numpy.ndarray'>
First item: /content/drive/MyDrive/processed_openpose_data/frames/1. loud/1. loud_temp_MVI_5177_0.jpg


In [ ]:
# Guess a path for a corresponding keypoint file
frame_path = data['train_frames'][0]
keypoint_path = frame_path.replace("frames", "keypoints").replace(".jpg", ".npy")

# Load it (if it exists)
import os

if os.path.exists(keypoint_path):
    keypoints = np.load(keypoint_path)
    print("Loaded keypoints shape:", keypoints.shape)
else:
    print("Keypoint file not found at:", keypoint_path)


Keypoint file not found at: /content/drive/MyDrive/processed_openpose_data/keypoints/1. loud/1. loud_temp_MVI_5177_0.npy


In [ ]:
import os
import glob

# Adjust this base path if needed
base_dir = "/content/drive/MyDrive/processed_openpose_data"

# Find all .npy files under the directory tree
npy_files = glob.glob(os.path.join(base_dir, "**", "*.npy"), recursive=True)
print(f"Found {len(npy_files)} .npy files.")
print(npy_files[:10])  # show the first few

# If you’re using OpenPose JSON output:
json_files = glob.glob(os.path.join(base_dir, "**", "*.json"), recursive=True)
print(f"Found {len(json_files)} .json files.")
print(json_files[:10])


Found 0 .npy files.
[]
Found 0 .json files.
[]


In [ ]:
import numpy as np
import cv2
import mediapipe as mp

# Initialize MediaPipe Holistic once
mp_holistic = mp.solutions.holistic

def extract_keypoints(frame_path):
    # Read & preprocess
    frame = cv2.imread(frame_path)
    if frame is None:
        print(f"Could not load image at {frame_path}")
        return None
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process with the same Holistic instance
    with mp_holistic.Holistic(static_image_mode=True) as holistic:
        results = holistic.process(image_rgb)

    kp = []

    # Pose: 33 landmarks × [x, y, z, visibility]
    if results.pose_landmarks:
        for lm in results.pose_landmarks.landmark:
            kp.extend([lm.x, lm.y, lm.z, lm.visibility])
    else:
        kp.extend([0.0] * 33 * 4)

    # Face: 468 landmarks × [x, y, z]
    if results.face_landmarks:
        for lm in results.face_landmarks.landmark:
            kp.extend([lm.x, lm.y, lm.z])
    else:
        kp.extend([0.0] * 468 * 3)

    # Left hand: 21 landmarks × [x, y, z]
    if results.left_hand_landmarks:
        for lm in results.left_hand_landmarks.landmark:
            kp.extend([lm.x, lm.y, lm.z])
    else:
        kp.extend([0.0] * 21 * 3)

    # Right hand: 21 landmarks × [x, y, z]
    if results.right_hand_landmarks:
        for lm in results.right_hand_landmarks.landmark:
            kp.extend([lm.x, lm.y, lm.z])
    else:
        kp.extend([0.0] * 21 * 3)

    return np.array(kp, dtype=np.float32)


In [ ]:
# Load metadata
meta = np.load("/content/drive/MyDrive/processed_openpose_data/dataset_meta.npz", allow_pickle=True)
train_frame_paths = meta['train_frames']
test_frame_paths  = meta['test_frames']

# Extract and save
os.makedirs(keypoint_save_dir, exist_ok=True)

train_keypoints = [extract_keypoints(p) for p in train_frame_paths]
train_keypoints = np.stack([kp for kp in train_keypoints if kp is not None])
np.save(os.path.join(keypoint_save_dir, 'train_keypoints.npy'), train_keypoints)

test_keypoints = [extract_keypoints(p) for p in test_frame_paths]
test_keypoints = np.stack([kp for kp in test_keypoints if kp is not None])
np.save(os.path.join(keypoint_save_dir, 'test_keypoints.npy'), test_keypoints)


KeyboardInterrupt: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load the .npz file
data = np.load("/content/drive/MyDrive/processed_openpose_data/dataset_meta.npz", allow_pickle=True)

train_frames = data['train_frames']
print("Train shape:", train_frames.shape)

# Choose a sample (1 frame)
sample = train_frames[0]

# Total length will tell us what kind of data it includes
print("Sample shape:", sample.shape)

# Try visualizing by assuming standard OpenPose-like layout:
# Let's guess it contains (pose: 25*3), or (pose + hands + face)
# We'll try splitting it manually if needed

# If sample is 1662 long, it's probably:
#  - 33 pose landmarks * 4 (x, y, z, visibility)
#  - 468 face landmarks * 3
#  - 21 hand landmarks * 3 each (left and right)

# Pose: 33*4
pose = sample[0:33*4].reshape(33, 4)
# Face: 468*3
face = sample[33*4:33*4 + 468*3].reshape(468, 3)
# Left hand: 21*3
lh_start = 33*4 + 468*3
lh = sample[lh_start:lh_start + 21*3].reshape(21, 3)
# Right hand: 21*3
rh = sample[lh_start + 21*3:].reshape(21, 3)

# Plotting
plt.figure(figsize=(10, 10))
plt.title("Sample Keypoints from train_frames")

def plot_points(points, color, label):
    xs = points[:, 0]
    ys = -points[:, 1]
    plt.scatter(xs, ys, label=label, color=color, s=10)

plot_points(pose, 'blue', 'Pose')
plot_points(face, 'green', 'Face')
plot_points(lh, 'red', 'Left Hand')
plot_points(rh, 'orange', 'Right Hand')

plt.legend()
plt.axis('equal')
plt.grid(True)
plt.show()


Train shape: (35819,)
Sample shape: ()


AttributeError: 'str' object has no attribute 'reshape'

In [ ]:

# Configuration
DATASET_PATH = "/content/drive/MyDrive/datauk.zip"  # Update this
OUTPUT_PATH = "/content/drive/MyDrive/new_processed_data"
IMG_SIZE = (368, 368)  # OpenPose standard input size
HEATMAP_SIZE = (46, 46)  # 1/8 of input size
SEQ_LENGTH = 16  # For temporal processing
os.makedirs(OUTPUT_PATH, exist_ok=True)

In [ ]:
import os
import cv2
import zipfile
from moviepy.editor import VideoFileClip
from tqdm import tqdm
import mediapipe as mp
import sys
from pathlib import Path

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False,
                   min_detection_confidence=0.5,
                   min_tracking_confidence=0.5)

def process_videos_to_openpose(video_dir, output_dir, frames_per_video=50):
    """Extracts frames and OpenPose keypoints from videos, handling nested zip structures"""
    os.makedirs(output_dir, exist_ok=True)
    all_frames = []
    all_keypoints = []
    processed_words = set()

    print(f"📂 Scanning: {video_dir}")
    sys.stdout.flush()

    def process_video_file(video_path, word_category=""):
        """Helper function to process individual video files"""
        try:
            clip = VideoFileClip(video_path)
            video_name = Path(video_path).stem

            # Create output subdirectory for word category
            word_output_dir = os.path.join(output_dir, word_category)
            os.makedirs(word_output_dir, exist_ok=True)

            frames = []
            keypoints = []

            for frame_idx, frame in enumerate(clip.iter_frames()):
                if frame_idx % max(1, int(clip.fps / frames_per_video)) == 0:
                    # Save frame
                    frame_path = os.path.join(word_output_dir,
                                            f"{word_category}_{video_name}_{frame_idx}.jpg")
                    success = cv2.imwrite(frame_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

                    if not success:
                        print(f"❌ Failed to save: {frame_path}")
                        continue

                    frames.append(frame_path)

                    # Check the frame type
                    print(f"Processing frame {frame_idx}: Shape={frame.shape}, Type={frame.dtype}")

                    # Get keypoints
                    results = pose.process(frame)
                    if results.pose_landmarks:
                        kps = []
                        for landmark in results.pose_landmarks.landmark:
                            kps.extend([landmark.x, landmark.y, landmark.visibility])
                        keypoints.append(kps)
                    else:
                        print(f"⚠️ No keypoints found for frame {frame_idx}")
                        keypoints.append([])

            clip.close()
            return frames, keypoints

        except Exception as e:
            print(f"⚠️ Error processing {video_path}: {str(e)}")
            return [], []

    if video_dir.endswith(".zip"):
        with zipfile.ZipFile(video_dir, 'r') as zip_ref:
            # Find all video files, preserving directory structure
            video_files = [f for f in zip_ref.namelist()
                         if f.lower().endswith(('.mp4', '.avi', '.mov'))]

            print(f"🎞 Found {len(video_files)} videos in zip")

            for video_file in tqdm(video_files, desc="Processing videos"):
                # Extract word category from path (assuming structure: words/word1/videos.mp4)
                path_parts = Path(video_file).parts
                word_category = path_parts[-2] if len(path_parts) > 1 else "unknown"

                if word_category not in processed_words:
                    print(f"\nProcessing word category: {word_category}")
                    processed_words.add(word_category)

                # Extract and process video
                with zip_ref.open(video_file) as video_data:
                    temp_path = f"temp_{Path(video_file).name}"
                    with open(temp_path, "wb") as temp_file:
                        temp_file.write(video_data.read())

                    frames, kps = process_video_file(temp_path, word_category)
                    all_frames.extend(frames)
                    all_keypoints.extend(kps)

                    # Clean up
                    if os.path.exists(temp_path):
                        os.remove(temp_path)
    else:
        # Process directory with word subfolders
        for root, dirs, files in os.walk(video_dir):
            video_files = [f for f in files if f.lower().endswith(('.mp4', '.avi', '.mov'))]

            if video_files:
                word_category = Path(root).name
                print(f"\nProcessing word category: {word_category}")

                for video_file in tqdm(video_files, desc=f"Processing {word_category}"):
                    video_path = os.path.join(root, video_file)
                    frames, kps = process_video_file(video_path, word_category)
                    all_frames.extend(frames)
                    all_keypoints.extend(kps)

    print(f"\n✅ Done. Extracted {len(all_frames)} frames and {len(all_keypoints)} keypoints sets.")
    print(f"Processed {len(processed_words)} word categories: {sorted(processed_words)}")

    return all_frames, all_keypoints

# Example usage:
# frames, keypoints = process_videos_to_openpose("dataset.zip", "output_frames")
# OR
# frames, keypoints = process_videos_to_openpose("dataset_folder", "output_frames")


In [ ]:


# Cell 6: Data Augmentation
def augment_data(image, heatmap):
    """Applies random augmentations"""
    # Random flip
    if np.random.rand() > 0.5:
        image = cv2.flip(image, 1)
        heatmap = np.flip(heatmap, axis=1)

    # Random brightness
    alpha = np.random.uniform(0.8, 1.2)
    image = cv2.convertScaleAbs(image, alpha=alpha)

    return image, heatmap


In [ ]:
!pip install -q numpy==1.23.5 opencv-python tqdm mediapipe tensorflow==2.12.0 moviepy gdown

# Optional: install unzip utility (only needed if you're downloading zip files)
!apt-get -qq install -y unzip

!pip install scikit-learn

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path

In [ ]:
import os
import cv2
import zipfile
import numpy as np
from pathlib import Path
import mediapipe as mp
from sklearn.model_selection import train_test_split

# ─── Configuration ──────────────────────────────────────────────────────────────
ZIP_FILE_PATH  = "/content/drive/MyDrive/datauk.zip"
EXTRACT_PATH   = "/content/drive/MyDrive/new_processed_data"
FRAMES_PER_VID = 50
# ────────────────────────────────────────────────────────────────────────────────

# ─── Constants ──────────────────────────────────────────────────────────────────
POSE_LM, FACE_LM, HAND_LM = 33, 468, 21
KP_LEN = POSE_LM * 4 + FACE_LM * 3 + HAND_LM * 3 * 2  # 1662
VIDEO_EXTS = {'.mp4', '.avi', '.mov', '.mkv', '.flv'}

# ─── Helper Functions ──────────────────────────────────────────────────────────
def extract_zip(zip_file, extract_to):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"✅ Extracted ZIP to: {extract_to}")


def extract_kps_from_frame(rgb_frame):
    holo = mp.solutions.holistic.Holistic(
        static_image_mode=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )
    res = holo.process(rgb_frame)
    kp = np.zeros(KP_LEN, dtype=np.float32)
    idx = 0

    # Pose
    if res.pose_landmarks:
        for lm in res.pose_landmarks.landmark:
            kp[idx:idx+4] = (lm.x, lm.y, lm.z, lm.visibility)
            idx += 4
    else:
        idx += POSE_LM * 4

    # Face
    if res.face_landmarks:
        for lm in res.face_landmarks.landmark:
            kp[idx:idx+3] = (lm.x, lm.y, lm.z)
            idx += 3
    else:
        idx += FACE_LM * 3

    # Hands
    for hand in (res.left_hand_landmarks, res.right_hand_landmarks):
        if hand:
            for lm in hand.landmark:
                kp[idx:idx+3] = (lm.x, lm.y, lm.z)
                idx += 3
        else:
            idx += HAND_LM * 3

    return kp


def process_video(video_path, output_dir):
    cap = cv2.VideoCapture(str(video_path))
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total <= 0:
        cap.release()
        return []

    indices = np.linspace(0, total - 1, FRAMES_PER_VID, dtype=int)
    out_dir = Path(output_dir) / video_path.parent.name
    out_dir.mkdir(parents=True, exist_ok=True)

    results = []
    for idx in indices:
        frame_file = out_dir / f"{video_path.stem}_{idx}.jpg"
        kp_file = out_dir / f"{video_path.stem}_{idx}_kps.npy"

        if frame_file.exists() and kp_file.exists():
            results.append((str(frame_file), str(kp_file)))
            continue

        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            continue

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        kp = extract_kps_from_frame(rgb)

        cv2.imwrite(str(frame_file), frame)
        np.save(str(kp_file), kp)
        results.append((str(frame_file), str(kp_file)))

    cap.release()
    return results

# ─── Main ───────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    # 1. Extract ZIP
    extract_zip(ZIP_FILE_PATH, EXTRACT_PATH)

    # 2. Gather video files
    video_files = [p for p in Path(EXTRACT_PATH).rglob("*") if p.suffix.lower() in VIDEO_EXTS]
    print(f"🎥 Found {len(video_files)} video files.")

    if not video_files:
        print("Error: No video files found. Check your ZIP content and path.")
        exit(1)

    # 3. Process videos sequentially
    all_data = []
    for video_path in video_files:
        all_data.extend(process_video(video_path, EXTRACT_PATH))

    if not all_data:
        print("Error: No frames or keypoints extracted.")
        exit(1)

    frame_paths, kp_paths = zip(*all_data)

    # 4. Train/test split by video folder
    video_names = [Path(fp).parent.name for fp in frame_paths]
    unique_vids = list(set(video_names))
    train_vids, test_vids = train_test_split(unique_vids, test_size=0.2, random_state=42)

    train_frames = [fp for fp, kp in zip(frame_paths, kp_paths) if Path(fp).parent.name in train_vids]
    test_frames  = [fp for fp, kp in zip(frame_paths, kp_paths) if Path(fp).parent.name in test_vids]

    train_kp =     [kp for fp, kp in zip(frame_paths, kp_paths) if Path(fp).parent.name in train_vids]
    test_kp =      [kp for fp, kp in zip(frame_paths, kp_paths) if Path(fp).parent.name in test_vids]

    # 5. Save metadata
    np.savez(Path(EXTRACT_PATH) / "dataset_meta.npz",
             train_frames=train_frames,
             test_frames=test_frames,
             train_kp=train_kp,
             test_kp=test_kp,
             input_shape=(FRAMES_PER_VID, 368, 368, 3),
             output_shape=(FRAMES_PER_VID, KP_LEN))

    print(f"✅ Extraction complete. Data and metadata saved in: {EXTRACT_PATH}")


KeyboardInterrupt: 

In [ ]:
import os
import zipfile
import numpy as np
import cv2
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor, as_completed
import mediapipe as mp
from sklearn.model_selection import train_test_split

# ─── Configuration ──────────────────────────────────────────────────────────────
ZIP_FILE_PATH  = "/content/drive/MyDrive/datauk.zip"
EXTRACT_PATH   = "/content/drive/MyDrive/new_processed_data"
FRAMES_PER_VID = 50
NUM_WORKERS    = 4   # Adjust to your available CPU cores
# ────────────────────────────────────────────────────────────────────────────────

# Precompute total keypoint length
POSE_LM, FACE_LM, HAND_LM = 33, 468, 21
KP_LEN = POSE_LM*4 + FACE_LM*3 + HAND_LM*3*2  # 1662

# MediaPipe model factory (each process will call this)
def make_holistic():
    mp_h = mp.solutions.holistic
    return mp_h.Holistic(static_image_mode=False,
                         min_detection_confidence=0.5,
                         min_tracking_confidence=0.5)

# Extract keypoints from a single frame (RGB ndarray)
def extract_kps_from_frame(holo, rgb_frame):
    res = holo.process(rgb_frame)
    kp = np.zeros(KP_LEN, dtype=np.float32)
    idx = 0

    # Pose (x,y,z,vis)
    if res.pose_landmarks:
        for lm in res.pose_landmarks.landmark:
            kp[idx:idx+4] = (lm.x, lm.y, lm.z, lm.visibility)
            idx += 4
    else:
        idx += POSE_LM*4

    # Face (x,y,z)
    if res.face_landmarks:
        for lm in res.face_landmarks.landmark:
            kp[idx:idx+3] = (lm.x, lm.y, lm.z)
            idx += 3
    else:
        idx += FACE_LM*3

    # Left & Right Hand (x,y,z each)
    for hand in (res.left_hand_landmarks, res.right_hand_landmarks):
        if hand:
            for lm in hand.landmark:
                kp[idx:idx+3] = (lm.x, lm.y, lm.z)
                idx += 3
        else:
            idx += HAND_LM*3

    return kp

# Process one video file: extract frames + keypoints
def process_single_video(video_path, frames_per_video=FRAMES_PER_VID):
    cap = cv2.VideoCapture(str(video_path))
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total <= 0:
        cap.release()
        return [], []

    # Determine frame indices to sample
    indices = np.linspace(0, total - 1, frames_per_video, dtype=int)
    frames, kps = [], []
    holo = make_holistic()

    for i in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame_bgr = cap.read()
        if not ret:
            continue

        # Convert and extract
        rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        kp = extract_kps_from_frame(holo, rgb)

        # Save frame image
        out_dir = Path(EXTRACT_PATH) / video_path.parent.name
        out_dir.mkdir(parents=True, exist_ok=True)
        out_img = out_dir / f"{video_path.stem}_{i}.jpg"
        cv2.imwrite(str(out_img), frame_bgr)

        frames.append(str(out_img))
        kps.append(kp)

    cap.release()
    return frames, kps

# Extract zip file
def extract_zip(zip_file, extract_to):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
        print(f"✅ Extracted ZIP file to {extract_to}")

# ── Main ───────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    # 1. Extract videos from zip file
    extract_zip(ZIP_FILE_PATH, EXTRACT_PATH)

    # 2. Gather all video paths (now that files are extracted)
    vid_paths = list(Path(EXTRACT_PATH).rglob("*.*"))
    vid_paths = [p for p in vid_paths if p.suffix.lower() in (".mp4", ".avi", ".mov", ".mkv", ".flv")]
    print(f"Found {len(vid_paths)} video files.")

    if not vid_paths:
        print("No valid video files found in the extracted directory.")
        exit()

    # 3. Parallel processing of videos to frames + keypoints
    all_frames, all_keypoints = [], []
    with ProcessPoolExecutor(NUM_WORKERS) as exe:
        futures = {exe.submit(process_single_video, vp): vp for vp in vid_paths}
        for fut in as_completed(futures):
            frames, kps = fut.result()
            all_frames.extend(frames)
            all_keypoints.extend(kps)

    print(f"Extracted {len(all_frames)} frames and {len(all_keypoints)} keypoint sets.")

    # 4. Train/test split by video directory name
    vids_by_cat = {}
    for fp, kp in zip(all_frames, all_keypoints):
        cat = Path(fp).parent.name
        vids_by_cat.setdefault(cat, []).append((fp, kp))

    cats = list(vids_by_cat)
    train_cats, test_cats = train_test_split(cats, test_size=0.2, random_state=42)

    train_data = [item for c in train_cats for item in vids_by_cat[c]]
    test_data  = [item for c in test_cats  for item in vids_by_cat[c]]

    # 5. Save .npy files
    np.save(Path(EXTRACT_PATH) / "train_keypoints.npy", np.stack([kp for _, kp in train_data]))
    np.save(Path(EXTRACT_PATH) / "test_keypoints.npy",  np.stack([kp for _, kp in test_data ]))
    np.savez(Path(EXTRACT_PATH) / "dataset_meta.npz",
             train_frames=[fp for fp, _ in train_data],
             test_frames =[fp for fp, _ in test_data],
             input_shape  = (FRAMES_PER_VID, 368, 368, 3),  # Example shape (adjust as needed)
             output_shape = (FRAMES_PER_VID, KP_LEN))

    print("✔️ Processing complete. Files saved under:", EXTRACT_PATH)


✅ Extracted ZIP file to /content/drive/MyDrive/new_processed_data
Found 730 video files.


BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [ ]:
import os
import shutil
from pathlib import Path
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor

# ========== CONFIG ==========
PROCESSED_PATH = Path("/content/drive/MyDrive/processed_openpose_data")  # Use local path, not Google Drive
OUTPUT_ROOT = Path("/content/drive/MyDrive/newopenpose_train_test_data")
TEST_SIZE = 0.2
SEED = 42

# Dummy augmentation (replace with your real augmenter)
class DummyAugmentor:
    def random_transform(self, img):
        return cv2.flip(img, 1)  # Simple horizontal flip

TRAIN_AUGMENTOR = DummyAugmentor()

# ========== MAIN ==========
def process_and_split():
    all_samples = []
    for root, _, files in os.walk(PROCESSED_PATH):
        if not files:
            continue
        category = Path(root).name
        for file in files:
            if file.endswith(('.jpg', '.png')):
                all_samples.append((Path(root) / file, category))

    print(f"Found {len(all_samples)} images in {PROCESSED_PATH}")

    file_paths = [str(p[0]) for p in all_samples]
    categories = [p[1] for p in all_samples]

    train_files, test_files = train_test_split(
        file_paths,
        test_size=TEST_SIZE,
        random_state=SEED,
        stratify=categories
    )

    print(f"Train: {len(train_files)} | Test: {len(test_files)}")

    train_dir = OUTPUT_ROOT / "train"
    test_dir = OUTPUT_ROOT / "test"
    train_dir.mkdir(parents=True, exist_ok=True)
    test_dir.mkdir(parents=True, exist_ok=True)

    def process_one(src_path, target_dir, augment):
        src = Path(src_path)
        category = src.parent.name
        dest_dir = target_dir / category
        dest_dir.mkdir(exist_ok=True)

        # Copy original image
        dest_file = dest_dir / src.name
        if not dest_file.exists():
            shutil.copy2(src, dest_file)

        # Augment
        if augment:
            img = cv2.imread(str(src))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            for i in range(1):  # 1 augmented image per original
                aug_img = TRAIN_AUGMENTOR.random_transform(img)
                aug_file = dest_dir / f"aug_{i}_{src.name}"
                cv2.imwrite(str(aug_file), cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR))

    def process_split_parallel(file_list, target_dir, augment=False):
        with ThreadPoolExecutor(max_workers=8) as executor:
            list(tqdm(executor.map(lambda f: process_one(f, target_dir, augment), file_list), total=len(file_list)))

    # Process both splits
    process_split_parallel(train_files, train_dir, augment=True)
    process_split_parallel(test_files, test_dir, augment=False)

    print("✅ Processing complete.")
    print(f"Train files: {sum(len(f) for _, _, f in os.walk(train_dir))}")
    print(f"Test files: {sum(len(f) for _, _, f in os.walk(test_dir))}")

# Run the processing
if __name__ == "__main__":
    process_and_split()


Found 45295 images in /content/drive/MyDrive/processed_openpose_data
Train: 36236 | Test: 9059


100%|██████████| 9059/9059 [07:45<00:00, 19.47it/s]


✅ Processing complete.
Train files: 72472
Test files: 9059


In [ ]:
import os
import hashlib
from pathlib import Path
from tqdm import tqdm

def compute_image_hash(image_path):
    with open(image_path, 'rb') as f:
        return hashlib.md5(f.read()).hexdigest()

def remove_duplicate_images(folder: Path):
    print(f"🔍 Scanning for duplicates in {folder}")
    seen_hashes = set()
    deleted = 0

    for root, _, files in os.walk(folder):
        for file in files:
            if file.endswith((".jpg", ".png")):
                file_path = Path(root) / file
                try:
                    img_hash = compute_image_hash(file_path)
                    if img_hash in seen_hashes:
                        os.remove(file_path)
                        deleted += 1
                    else:
                        seen_hashes.add(img_hash)
                except Exception as e:
                    print(f"⚠️ Error reading {file_path}: {e}")

    print(f"✅ Done. Removed {deleted} duplicate frames from {folder}.")

# Run this only after splitting is done
if __name__ == "__main__":
    train_folder = Path("/content/drive/MyDrive/newopenpose_train_test_data/train")
    test_folder  = Path("/content/drive/MyDrive/newopenpose_train_test_data/test")

    remove_duplicate_images(train_folder)
    remove_duplicate_images(test_folder)


🔍 Scanning for duplicates in /content/drive/MyDrive/newopenpose_train_test_data/train


In [ ]:
import os
import shutil
import cv2
import numpy as np
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# ========== CONFIG ==========
PROCESSED_PATH = Path("/content/drive/MyDrive/processed_openpose_data")  # Existing frames directory
OUTPUT_ROOT    = Path("/content/drive/MyDrive/feature_train_test_data")  # Destination root
MOTION_THRESHOLD = 500  # adjust sensitivity
MAX_WORKERS = 8  # threads for parallel processing

# Dummy augmentor (replace with your own)
class DummyAugmentor:
    def random_transform(self, img):
        return cv2.flip(img, 1)

TRAIN_AUGMENTOR = DummyAugmentor()

# ========== FUNCTIONS ==========

def has_motion(prev_frame: np.ndarray, curr_frame: np.ndarray, threshold: float = MOTION_THRESHOLD) -> bool:
    """
    Compare two frames and return True if motion exceeds threshold.
    """
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
    diff = cv2.absdiff(prev_gray, curr_gray)
    return np.sum(diff) > threshold


def save_if_motion(prev_frame: np.ndarray, frame_path: Path, dest_dir: Path, augment: bool = False) -> np.ndarray:
    """
    Copy frame to dest_dir if motion detected, optionally augment.
    Returns current frame if saved (for next comparison), else returns prev_frame.
    """
    img = cv2.imread(str(frame_path))
    if img is None:
        return prev_frame

    if prev_frame is None or has_motion(prev_frame, img):
        # ensure category directory
        cat_dir = dest_dir / frame_path.parent.name
        cat_dir.mkdir(parents=True, exist_ok=True)
        # copy original frame
        dst = cat_dir / frame_path.name
        shutil.copy2(frame_path, dst)

        # augment if requested
        if augment:
            rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            aug = TRAIN_AUGMENTOR.random_transform(rgb)
            aug_bgr = cv2.cvtColor(aug, cv2.COLOR_RGB2BGR)
            aug_name = f"aug_{frame_path.name}"
            cv2.imwrite(str(cat_dir / aug_name), aug_bgr)

        return img

    # no motion: skip saving
    return prev_frame


def process_files_with_motion(file_list: list, dest_root: Path, augment: bool = False):
    """
    Process a list of frame file paths:
    - Filters out static frames via motion detection.
    - Copies frames with motion to dest_root.
    - Optionally augments images when augment=True.
    """
    prev_by_cat = {}
    # ensure root exists
    dest_root.mkdir(parents=True, exist_ok=True)

    def _process(path_str):
        frame_path = Path(path_str)
        cat = frame_path.parent.name
        prev = prev_by_cat.get(cat)
        new_prev = save_if_motion(prev, frame_path, dest_root, augment)
        prev_by_cat[cat] = new_prev

    # Use ThreadPoolExecutor for speed
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        list(tqdm(executor.map(_process, file_list), total=len(file_list), desc=("Augment" if augment else "Filter")))
# if __name__ == '__main__':
#     # Example: assume you have two lists of frame paths already
#     train_frame_list = [str(p) for p in (PROCESSED_PATH / 'category1').glob('*.jpg')]
#     test_frame_list  = [str(p) for p in (PROCESSED_PATH / 'category2').glob('*.jpg')]

#     # Process train (with augmentation)
#     process_files_with_motion(train_frame_list, OUTPUT_ROOT / 'train', augment=True)
#     # Process test (without augmentation)
#     process_files_with_motion(test_frame_list, OUTPUT_ROOT / 'test', augment=False)

#     print("✅ Motion filtering and copying complete.")
if __name__ == '__main__':
    frames_root = PROCESSED_PATH / 'frames'

    # Get all frame paths grouped by category
    train_frame_list = []
    test_frame_list = []

    for category_dir in frames_root.iterdir():
        if category_dir.is_dir():
            all_images = sorted(category_dir.glob("*.jpg"))
            split_idx = int(len(all_images) * 0.8)
            train_frame_list.extend([str(p) for p in all_images[:split_idx]])
            test_frame_list.extend([str(p) for p in all_images[split_idx:]])

    print(f"Train frames: {len(train_frame_list)} | Test frames: {len(test_frame_list)}")

    # Process train (with augmentation)
    process_files_with_motion(train_frame_list, OUTPUT_ROOT / 'train', augment=True)

    # Process test (without augmentation)
    process_files_with_motion(test_frame_list, OUTPUT_ROOT / 'test', augment=False)

    print("✅ Motion filtering and copying complete.")


Train frames: 36219 | Test frames: 9076


Filter: 100%|██████████| 9076/9076 [12:58<00:00, 11.66it/s]

✅ Motion filtering and copying complete.
